# Explaining 

│   ├ ymdhm : 년월일시분

│   ├ swl : 팔당댐 현재수위 (단위: El.m)

│   ├ inf : 팔당댐 유입량 (단위: m^3/s)

│   ├ sfw : 팔당댐 저수량 (단위: 만m^3)

│   ├ ecpc : 팔당댐 공용량 (단위: 백만m^3)

│   ├ tototf : 총 방류량 (단위: m^3/s)

│   ├ tide_level : 강화대교 조위 (단위: cm)

│   ├ wl_1018662 : 청담대교 수위 (단위: cm)

│   ├ fw_1018662 : 청담대교 유량 (단위: m^3/s)

│   ├ wl_1018680 : 잠수교 수위 (단위: cm)

│   ├ fw_1018680 : 잠수교 유량 (단위: m^3/s)

│   ├ wl_1018683 : 한강대교 수위 (단위: cm)

│   ├ fw_1018683 : 한강대교 유량 (단위: m^3/s)

│   ├ wl_1019630 : 행주대교 수위 (단위: cm)

│   └ fw_1019630 : 행주대교 유량 (단위: m^3/s)

│   ├ rf_10184100 : 대곡교 강수량

│   ├ rf_10184110 : 진관교 강수량

│   └ rf_10184140 : 송정동 강수량

# Setting

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

In [4]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [5]:
from sklearn.impute import KNNImputer

# Load Data

In [6]:
def read_csv_by_dir(path, index_col=None):

    df_raw = pd.DataFrame()

    for files in os.listdir(path):

        if files.endswith('.csv'):

            df = pd.read_csv('/'.join([path,files]), index_col=index_col)

        df_raw = pd.concat((df_raw,df),axis=0)
        
    return df_raw

In [7]:
# path = '/content/drive/MyDrive/시계열/팔당댐'
path = '/Users/namwoo/Desktop/팔당댐'

df_rf_raw = read_csv_by_dir('/'.join([path,'rf_data']), index_col=None)

df_water_raw = read_csv_by_dir('/'.join([path,'water_data']), index_col=None)

submission_raw = pd.read_csv('/'.join([path,'sample_submission.csv']), index_col=0)

In [8]:
# raw_data 보존하기
df_rf = df_rf_raw.copy()
df_rf.name = "rain_data"

df_water = df_water_raw.copy()
df_water.name = "water_data"

submission = submission_raw.copy()
submission.name = "submission"

In [9]:
display(df_rf_raw.head(),
        df_water.head(),
        submission.head())

,ymdhm,rf_10184100,rf_10184110,rf_10184140
0,2022-05-01 00:00,0.0,0.0,0.0
1,2022-05-01 00:10,0.0,0.0,0.0
2,2022-05-01 00:20,0.0,0.0,0.0
3,2022-05-01 00:30,0.0,0.0,0.0
4,2022-05-01 00:40,0.0,0.0,0.0


,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
0,2021-05-01 00:00,24.93,0.0,208.88,35.12,138.0,311.0,361.7,905.32,346.2,NaN,340.0,983.18,301.3,895.58
1,2021-05-01 00:10,24.93,0.0,208.88,35.12,138.0,297.0,356.7,856.07,338.2,NaN,335.0,1285.80,308.3,1006.88
2,2021-05-01 00:20,24.92,0.0,208.52,35.48,138.0,283.0,347.7,770.98,331.2,NaN,331.0,1381.43,313.3,1090.45
3,2021-05-01 00:30,24.92,0.0,208.52,35.48,151.0,271.0,337.7,681.80,327.2,NaN,329.0,1359.95,315.3,1124.83
4,2021-05-01 00:40,24.92,0.0,208.52,35.48,151.0,259.0,328.7,606.35,323.2,NaN,327.0,1267.65,316.3,1142.22


,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-06-01 00:00,0,0,0,0
2022-06-01 00:10,0,0,0,0
2022-06-01 00:20,0,0,0,0
2022-06-01 00:30,0,0,0,0
2022-06-01 00:40,0,0,0,0


In [10]:
print(df_rf.shape,
        df_water.shape,
        submission.shape)

(276336, 4) (276336, 15) (6912, 4)


# Data preprocessing

## 날짜, 시간 columns 생성

In [11]:
def day_time_split(df):

  col_year = []
  col_month = []
  col_week = []

  col_hour = []
  # col_minute = []

  for info in tqdm(df['ymdhm']):

    list_info = info.split()
    date = list_info[0]
    time = list_info[-1]

    list_date = date.split('-')
    year = list_date[0]
    month = list_date[1]
    day = list_date[-1]

    col_year.append(year)
    col_month.append(month)

    day = int(day)

    if day <= 7:
      week = 1

    elif day <=14:
      week = 2
    
    elif day <= 21:
      week = 3
    
    elif day <= 28:
      week = 4
    
    else:
      week = 5
  
    col_week.append(str(week))

    list_time = time.split(':')
    hour = list_time[0]
    # minute = list_time[-1]

    col_hour.append(hour)
    # col_minute.append(int(minute))

  df['year'] = col_year
  df['month'] = col_month
  df['week'] = col_week

  df['hour'] = col_hour
  #ㅍdf['minute'] = col_minute

  df.set_index('ymdhm', drop=True, inplace=True)

  return df

In [12]:
df_rf = day_time_split(df_rf)
df_water = day_time_split(df_water)

  0%|          | 0/276336 [00:00<?, ?it/s]

  0%|          | 0/276336 [00:00<?, ?it/s]

## index format 변경

In [13]:
def index_to_datetime(df,format):

    df.index = pd.to_datetime(df.index, format=format)

    return df

In [14]:
df_rf

,rf_10184100,rf_10184110,rf_10184140,year,month,week,hour
ymdhm,,,,,,,
2022-05-01 00:00,0.0,0.0,0.0,2022,05,1,00
2022-05-01 00:10,0.0,0.0,0.0,2022,05,1,00
2022-05-01 00:20,0.0,0.0,0.0,2022,05,1,00
2022-05-01 00:30,0.0,0.0,0.0,2022,05,1,00
2022-05-01 00:40,0.0,0.0,0.0,2022,05,1,00
...,...,...,...,...,...,...,...
2013-10-31 23:10,0.0,0.0,0.0,2013,10,5,23
2013-10-31 23:20,0.0,0.0,0.0,2013,10,5,23
2013-10-31 23:30,0.0,0.0,0.0,2013,10,5,23


In [15]:
df_rf=index_to_datetime(df=df_rf,format='%Y-%m-%d %H:%M')
df_water=index_to_datetime(df=df_water,format='%Y-%m-%d %H:%M')
submission=index_to_datetime(df=submission,format='%Y-%m-%d %H:%M')

In [16]:
# 정렬
df_rf.sort_index(inplace=True)
df_water.sort_index(inplace=True)
submission.sort_index(inplace=True)

In [17]:
def check_datetime(df):

    print(df.name)
    print(df.select_dtypes('datetime64[ns]').head(1).index[0])
    print(df.select_dtypes('datetime64[ns]').tail(1).index[0])
    
    return None

check_datetime(df_rf)
check_datetime(df_water)
check_datetime(submission)

rain_data
2012-05-01 00:00:00
2022-07-18 23:50:00
water_data
2012-05-01 00:00:00
2022-07-18 23:50:00
submission
2022-06-01 00:00:00
2022-07-18 23:50:00


## Meraging Data

In [18]:
df_water.drop(columns=['year', 'month', 'week', 'hour'], inplace=True)
data = pd.concat((df_rf, df_water),axis=1)

In [ ]:
for col, null in zip(data.columns, data.isnull().sum()):

  percent = (null/data.shape[0]) * 100

  print('{}의 NULL 차지 비중 : {:.3} %'.format(col, percent))

In [19]:
data = pd.get_dummies(data)


In [20]:
for col, null in zip(data.columns, data.isnull().sum()):

  percent = (null/data.shape[0]) * 100

  print('{}의 NULL 차지 비중 : {:.3} %'.format(col, percent))

rf_10184100의 NULL 차지 비중 : 0.0 %
rf_10184110의 NULL 차지 비중 : 0.0 %
rf_10184140의 NULL 차지 비중 : 0.0 %
swl의 NULL 차지 비중 : 0.269 %
inf의 NULL 차지 비중 : 0.269 %
sfw의 NULL 차지 비중 : 0.269 %
ecpc의 NULL 차지 비중 : 0.269 %
tototf의 NULL 차지 비중 : 0.269 %
tide_level의 NULL 차지 비중 : 1.78 %
wl_1018662의 NULL 차지 비중 : 0.0214 %
fw_1018662의 NULL 차지 비중 : 5.93 %
wl_1018680의 NULL 차지 비중 : 0.0214 %
fw_1018680의 NULL 차지 비중 : 71.2 %
wl_1018683의 NULL 차지 비중 : 0.0214 %
fw_1018683의 NULL 차지 비중 : 0.463 %
wl_1019630의 NULL 차지 비중 : 0.0214 %
fw_1019630의 NULL 차지 비중 : 0.0214 %
year_2012의 NULL 차지 비중 : 0.0 %
year_2013의 NULL 차지 비중 : 0.0 %
year_2014의 NULL 차지 비중 : 0.0 %
year_2015의 NULL 차지 비중 : 0.0 %
year_2016의 NULL 차지 비중 : 0.0 %
year_2017의 NULL 차지 비중 : 0.0 %
year_2018의 NULL 차지 비중 : 0.0 %
year_2019의 NULL 차지 비중 : 0.0 %
year_2020의 NULL 차지 비중 : 0.0 %
year_2021의 NULL 차지 비중 : 0.0 %
year_2022의 NULL 차지 비중 : 0.0 %
month_05의 NULL 차지 비중 : 0.0 %
month_06의 NULL 차지 비중 : 0.0 %
month_07의 NULL 차지 비중 : 0.0 %
month_08의 NULL 차지 비중 : 0.0 %
month_09의 NULL 차지 비중 : 0.

# Train / Test Split

In [21]:
Y = df_water.loc[:,submission.columns]
X = data.drop(submission.columns,axis=1)

In [22]:
# data와 target 하나 밀어주기 (과거데이터를 사용해야 함으로)
_Y = Y.reset_index(drop=True)
_X = X.reset_index(drop=True)

_X.index += 1

tot=pd.concat((_X, _Y),axis=1)
tot=tot.sort_index()

tot=tot.iloc[1:-1]

Y = tot.loc[:,submission.columns]
X = tot.drop(submission.columns,axis=1)

In [23]:
train_Y = Y.iloc[:-len(submission),:]
test_Y = Y.iloc[-len(submission):,:]

train_X = X.iloc[:-len(submission),:]
test_X = X.iloc[-len(submission):,:]

## 결측치 제거

선형 보간법

In [24]:
train_Y = train_Y.fillna(method='ffill')
test_Y = test_Y.fillna(method='ffill')
train_X = train_X.fillna(method='ffill')
test_X = test_X.fillna(method='ffill')

In [25]:
# train_X['fw_1018680'] = [0.0] * train_X.shape[0]
# test_X['fw_1018680'] = [0.0] * test_X.shape[0]

In [26]:
!pip install catboost

In [27]:
!pip install pactools

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold

from sklearn.multioutput import MultiOutputRegressor
from catboost import Pool, CatBoostRegressor
import catboost as cb
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
from pactools.grid_search import GridSearchCVProgressBar
import multiprocessing

In [29]:
kfold = KFold(n_splits=5, shuffle=True)

# Light GBM

In [30]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV

In [36]:
pipe_model = Pipeline([('reg', MultiOutputRegressor(lgb.LGBMRegressor()))])

param_grid={"reg__estimator__n_estimators": [100],
          "reg__estimator__learning_rate": [0.1],
          "reg__estimator__max_depth": [8],
          "reg__estimator__bagging_fraction":[0.1],
          "reg__estimator__feature_fraction":[0.8],
          "reg__estimator__min_data_in_leaf":[100],
          "reg__estimator__num_leaves":[100],
         }

# model = lgb.LGBMRegressor()
# multilabel_model = MultiOutputRegressor(model)

grid_model = (GridSearchCVProgressBar(estimator=pipe_model,
                                      param_grid=param_grid,
                                      cv=3,
                                      scoring='neg_root_mean_squared_error',
                                      n_jobs=multiprocessing.cpu_count()-1,
                                      verbose=10))



In [37]:
grid_model.fit(train_X, train_Y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8


GridSearchCVProgressBar(cv=3,
                        estimator=Pipeline(steps=[('reg',
                                                   MultiOutputRegressor(estimator=LGBMRegressor()))]),
                        n_jobs=7,
                        param_grid={'reg__estimator__bagging_fraction': [0.1],
                                    'reg__estimator__feature_fraction': [0.8],
                                    'reg__estimator__learning_rate': [0.1],
                                    'reg__estimator__max_depth': [8],
                                    'reg__estimator__min_data_in_leaf': [100],
                                    'reg__estimator__n_estimators': [100],
                                    'reg__estimator__num_leaves': [100]},
                        scoring='neg_root_mean_squared_error', verbose=10)

In [38]:
print(grid_model.best_estimator_)

Pipeline(steps=[('reg',
                 MultiOutputRegressor(estimator=LGBMRegressor(bagging_fraction=0.1,
                                                              feature_fraction=0.8,
                                                              max_depth=8,
                                                              min_data_in_leaf=100,
                                                              num_leaves=100)))])


In [39]:
model=grid_model.best_estimator_
model.fit(train_X,train_Y)

y_pred=model.predict(test_X)

submission_raw.iloc[:,:] = y_pred

In [40]:
submission_raw.to_csv('lgbm4.csv')